# Binary classification benchmark

This notebook compares binary classification models from various libraries. The dataset used is the [Higgs dataset](https://archive.ics.uci.edu/ml/datasets/HIGGS).

In [3]:
%load_ext watermark
%watermark --python --machine --packages creme,keras,skgarden,sklearn,tensorflow,torch --datename

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Mon Oct 07 2019 

CPython 3.7.4
IPython 7.4.0

creme 0.3.0
keras 2.2.4
skgarden 0.1.2
sklearn 0.21.3
tensorflow 1.14.0
torch 1.1.0

compiler   : GCC 7.3.0
system     : Linux
release    : 5.2.10-arch1-1-ARCH
machine    : x86_64
processor  : 
CPU cores  : 8
interpreter: 64bit


In [4]:
from creme import compose
from creme import feature_extraction
from creme import linear_model
from creme import metrics
from creme import neighbors
from creme import optim
from creme import preprocessing
from creme import stats
from creme import stream
from creme import tree
from keras import layers
from keras import models
from keras import optimizers
import numpy as np
import skgarden
from sklearn import linear_model as sk_linear_model
import torch

%run utils.py
%run wrappers.py

In [6]:
n_features = 28

class PyTorchNet(torch.nn.Module):
    
    def __init__(self, n_features):
        super().__init__()
        self.linear = torch.nn.Linear(n_features, 1)
        self.sigmoid = torch.nn.Sigmoid()
        torch.nn.init.constant_(self.linear.weight, 0)
        torch.nn.init.constant_(self.linear.bias, 0)
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
torch_model = PyTorchNet(n_features=n_features)

# Keras
inputs = layers.Input(shape=(n_features,))
predictions = layers.Dense(1, activation='sigmoid', kernel_initializer='zeros', bias_initializer='zeros')(inputs)
keras_model = models.Model(inputs=inputs, outputs=predictions)
keras_model.compile(optimizer=optimizers.SGD(), loss='binary_crossentropy')

names = [
    'is_signal',
    'lepton pT', 'lepton eta','lepton phi',
    'missing energy magnitude', 'missing energy phi', 'jet 1 pt',
    'jet 1 eta', 'jet 1 phi', 'jet 1 b-tag',
    'jet 2 pt', 'jet 2 eta', 'jet 2 phi',
    'jet 2 b-tag', 'jet 3 pt', 'jet 3 eta',
    'jet 3 phi', 'jet 3 b-tag', 'jet 4 pt',
    'jet 4 eta', 'jet 4 phi', 'jet 4 b-tag',
    'm_jj', 'm_jjj', 'm_lv', 'm_jlv',
    'm_bb', 'm_wbb', 'm_wwbb'
]
converters = {name: float for name in names}
converters['is_signal'] = lambda x: bool(float(x))


import itertools

n = 100_000 # 11_000_000

get_X_y = lambda: itertools.islice(stream.iter_csv('../HIGGS.csv.gz', names=names, target_name='is_signal', converters=converters), n)

results = benchmark(
    get_X_y=get_X_y,
    n=n,
    get_pp=preprocessing.StandardScaler,
    models=[
        ('creme', 'LogisticRegression', linear_model.LogisticRegression()),
        ('creme', 'PAClassifier', linear_model.PAClassifier()),
        ('creme', 'DecisionTreeClassifier', tree.DecisionTreeClassifier()),
        ('creme', 'RandomForestClassifier', tree.RandomForestClassifier()),
        ('Keras on Tensorflow (CPU)', 'Dense', KerasBinaryClassifier(keras_model)),
        ('PyTorch (CPU)', 'Linear', PyTorchBinaryClassifier(
            network=torch_model,
            loss_fn=torch.nn.BCELoss(),
            optimizer=torch.optim.SGD(torch_model.parameters(), lr=0.01)
        )),
        ('scikit-learn', 'SGDClassifier', ScikitLearnClassifier(
            sk_linear_model.SGDClassifier(loss='log'),
            classes=[False, True]
        )),
        ('scikit-learn', 'PassiveAggressiveClassifier', ScikitLearnClassifier(
            sk_linear_model.PassiveAggressiveClassifier(),
            classes=[False, True]
        )),
        ('scikit-garden', 'MondrianTreeClassifier', ScikitLearnClassifier(
            skgarden.MondrianTreeClassifier(random_state=42),
            classes=np.array([False, True])
        )),
        ('scikit-garden', 'MondrianForestClassifier', ScikitLearnClassifier(
            skgarden.MondrianForestClassifier(random_state=42),
            classes=np.array([False, True])
        ))
    ],
    get_metric=metrics.Accuracy
)

W1007 14:18:04.004729 140532481337152 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1007 14:18:04.015311 140532481337152 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [7]:
results

,Library,Model,Accuracy,Fit time,Average fit time,Predict time,Average predict time
0,creme,LogisticRegression,0.61810,"2s, 607ms, 937μs, 564ns","26μs, 79ns","1s, 78ms, 473μs, 18ns","10μs, 785ns"
1,creme,PAClassifier,0.55009,"3s, 551ms, 273μs, 230ns","35μs, 513ns","2s, 252ms, 867μs, 34ns","22μs, 529ns"
2,creme,DecisionTreeClassifier,0.64663,"35s, 680ms, 385μs, 243ns","356μs, 804ns","1s, 507ms, 671μs, 262ns","15μs, 77ns"
3,creme,RandomForestClassifier,0.65915,"6m, 37s, 287ms, 767μs, 351ns","3ms, 972μs, 878ns","20s, 846ms, 87μs, 174ns","208μs, 461ns"
4,Keras on Tensorflow (CPU),Dense,0.61840,"46s, 354ms, 746μs, 18ns","463μs, 547ns","53s, 406ms, 330μs, 212ns","534μs, 63ns"
5,PyTorch (CPU),Linear,0.61840,"1m, 32s, 681ms, 67μs, 402ns","926μs, 811ns","1m, 2s, 155ms, 741μs, 452ns","621μs, 557ns"
6,scikit-learn,SGDClassifier,0.56161,"42s, 42ms, 496μs, 787ns","420μs, 425ns","11s, 670ms, 182μs, 924ns","116μs, 702ns"
7,scikit-learn,PassiveAggressiveClassifier,0.55009,"39s, 882ms, 226μs, 530ns","398μs, 822ns","11s, 453ms, 716μs, 995ns","114μs, 537ns"
8,scikit-garden,MondrianTreeClassifier,0.53875,"1m, 26s, 417ms, 227μs, 644ns","864μs, 172ns","20s, 806ms, 447μs, 91ns","208μs, 64ns"
9,scikit-garden,MondrianForestClassifier,0.60061,"16m, 17s, 391ms, 572μs, 273ns","9ms, 773μs, 916ns","2m, 3s, 370ms, 203μs, 352ns","1ms, 233μs, 702ns"


In [1]:
results

NameError: name 'results' is not defined